# Use LightGBM to do predictions in LightBBB dataset and compare with GNN

## Import packages and configs

In [2]:
import os

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import lightgbm as lgb
from sklearn.metrics import (
    roc_auc_score, 
    average_precision_score, 
    recall_score, 
    precision_score, 
    f1_score, 
    accuracy_score
)
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit import RDLogger

from splitters import scaffold_split_from_smiles

In [3]:
RDLogger.DisableLog("rdApp.*")

## Readin training dataset
Files in LightBBB dataset:
* y_test_indices.csv: contains 7162 BBB compound SMILES and permeability of each compound is in binary values (0 for non-permeable and 1 for permeable)
* datasetNormalizedDescrs.csv: contains 1119 2D molecular descriptors of 7162 compounds retrieved from Dragon software.
* y_indices_external.csv: contains 74 compound SMILIES and their permeability class in 0 and 1.    
* external_dataset.csv: contains 2D molecular descriptors of 74 compounds.  


Dataset files y_test_indices.csv and datasetNormalizedDescrs.csv used in model training while data in y_indices_external.csv and external_dataset.csv used for external test dataset model validation.

In [4]:
ds = pd.read_csv("dataset/lightbbb/raw/y_test_indices.csv")
descriptors = pd.read_csv("dataset/lightbbb/raw/datasetNormalizedDescrs.csv", index_col=0)
test_ds = pd.read_csv("dataset/lightbbb/raw/y_indices_external.csv")
test_des = pd.read_csv("dataset/lightbbb/raw/external_dataset.csv", index_col=0)

## Scaffold splitting using the method from Hu's work (ContextPred)

In [4]:
train_idx, val_idx, _ = scaffold_split_from_smiles(
    ds["smiles"],
    frac_train=0.9,
    frac_valid=0.1,
    frac_test=0
)

## Initialize LightGBM Dataset objects and setup parameters for training

In [7]:
training_data = lgb.Dataset(descriptors.iloc[train_idx].to_numpy(), label=ds.iloc[train_idx]["BBclass"])
validation_data = lgb.Dataset(descriptors.iloc[val_idx].to_numpy(), label=ds.iloc[val_idx]["BBclass"])
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = ['auc', 'average_precision']
num_round = 100

## Training

In [8]:
bst = lgb.train(param, training_data, num_round, valid_sets=[validation_data])

[LightGBM] [Info] Number of positive: 5045, number of negative: 1381
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031848 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 53610
[LightGBM] [Info] Number of data points in the train set: 6426, number of used features: 912
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.785092 -> initscore=1.295590
[LightGBM] [Info] Start training from score 1.295590
[1]	valid_0's auc: 0.840985	valid_0's average_precision: 0.820778
[2]	valid_0's auc: 0.859371	valid_0's average_precision: 0.850939
[3]	valid_0's auc: 0.866919	valid_0's average_precision: 0.869303
[4]	valid_0's auc: 0.865688	valid_0's average_precision: 0.866429
[5]	valid_0's auc: 0.865609	valid_0's average_precision: 0.864926
[6]	valid_0's auc: 0.862733	valid_0's average_precision: 0.850941
[7]	valid_0's auc: 0.865815	valid_0's average_precision: 0.860234
[8]	valid_0's auc: 0.867243	valid_0's averag

In [11]:
ypred = bst.predict(test_des.to_numpy())
ytrue = test_ds["BBclass"].to_numpy()
roc = roc_auc_score(ytrue, ypred)
ap = average_precision_score(ytrue, ypred)
acc = accuracy_score(ytrue, np.round(ypred))
precision = precision_score(ytrue, np.round(ypred), average="binary")
recall = recall_score(ytrue, np.round(ypred), average="binary")
specificity = recall_score(1-ytrue, 1-np.round(ypred), average="binary")
f1 = f1_score(ytrue, np.round(ypred), average="binary")

In [29]:
results = f"""Testing results:
    roc_auc:\t\t{roc:.6f}
    ap:\t\t\t{ap:.6f}
    accuracy:\t\t{acc:.6f}
    precision:\t\t{precision:.6f}
    recall:\t\t{recall:.6f}
    specificity:\t{specificity:.6f}
    f1:\t\t\t{f1:.6f}
"""
print(results)

Testing results:
    roc_auc:		0.947253
    ap:			0.935167
    accuracy:		0.918919
    precision:		0.902439
    recall:		0.948718
    specificity:	0.885714
    f1:			0.925000



## Use scaffold splitting to get testing set

In [12]:
train_idx, val_idx, test_idx = scaffold_split_from_smiles(
    ds["smiles"],
    frac_train=0.8,
    frac_valid=0.1,
    frac_test=0.1
)

training_data = lgb.Dataset(descriptors.iloc[train_idx].to_numpy(), label=ds.iloc[train_idx]["BBclass"])
validation_data = lgb.Dataset(descriptors.iloc[val_idx].to_numpy(), label=ds.iloc[val_idx]["BBclass"])
testing_data = descriptors.iloc[test_idx].to_numpy()
ytrue = ds.iloc[test_idx]["BBclass"].to_numpy()

rand_seed = 0
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = ['auc', 'average_precision']
param["feature_fraction_seed"] = rand_seed
param["extra_seed"] = rand_seed
param["drop_seed"] = rand_seed

num_round = 100

In [13]:
bst = lgb.train(param, training_data, num_round, valid_sets=[validation_data])

[LightGBM] [Info] Number of positive: 4561, number of negative: 1151
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032628 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 52222
[LightGBM] [Info] Number of data points in the train set: 5712, number of used features: 895
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.798494 -> initscore=1.376911
[LightGBM] [Info] Start training from score 1.376911
[1]	valid_0's auc: 0.893281	valid_0's average_precision: 0.917878
[2]	valid_0's auc: 0.908651	valid_0's average_precision: 0.92715
[3]	valid_0's auc: 0.913636	valid_0's average_precision: 0.933014
[4]	valid_0's auc: 0.912082	valid_0's average_precision: 0.926501
[5]	valid_0's auc: 0.913587	valid_0's average_precision: 0.935842
[6]	valid_0's auc: 0.914274	valid_0's average_precision: 0.937894
[7]	valid_0's auc: 0.918038	valid_0's average_precision: 0.939732
[8]	valid_0's auc: 0.914162	valid_0's average

In [14]:
ypred = bst.predict(testing_data)

roc = roc_auc_score(ytrue, ypred)
ap = average_precision_score(ytrue, ypred)
acc = accuracy_score(ytrue, np.round(ypred))
precision = precision_score(ytrue, np.round(ypred), average="binary")
recall = recall_score(ytrue, np.round(ypred), average="binary")
specificity = recall_score(1-ytrue, 1-np.round(ypred), average="binary")
f1 = f1_score(ytrue, np.round(ypred), average="binary")

In [15]:
results = f"""Testing results:
    roc_auc:\t\t{roc:.6f}
    ap:\t\t\t{ap:.6f}
    accuracy:\t\t{acc:.6f}
    precision:\t\t{precision:.6f}
    recall:\t\t{recall:.6f}
    specificity:\t{specificity:.6f}
    f1:\t\t\t{f1:.6f}
"""
print(results)

Testing results:
    roc_auc:		0.867682
    ap:			0.876543
    accuracy:		0.802797
    precision:		0.762397
    recall:		0.934177
    specificity:	0.640625
    f1:			0.839590

